In [12]:
!git clone https://github.com/pachocamacho1990/datasets


fatal: destination path 'datasets' already exists and is not an empty directory.


In [13]:
import pandas as pd
import numpy as np
import nltk
import re
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk import word_tokenize
import random
from sklearn.model_selection import  train_test_split

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [14]:
df = pd.read_csv('datasets/email/csv/spam-apache.csv', names=['clase','contenido'])

In [9]:
df.head()

,clase,contenido
0,-1,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr..."
1,1,> Russell Turpin:\n> > That depends on how the...
2,-1,Help wanted. We are a 14 year old fortune 500...
3,-1,Request A Free No Obligation Consultation!\nAc...
4,1,Is there a way to look for a particular file o...


In [ ]:
df['tokens'] = df['contenido'].apply(lambda x: word_tokenize(x))
df.head()

In [ ]:
df['tokens'][0][2900:]

In [ ]:
all_words = nltk.FreqDist([w for tokenlist in df['tokens'].values for w in tokenlist])

In [ ]:
top_words = all_words.most_common(200)
top_words

In [ ]:
def documento_atributos(document):
  document_words = set(document)
  atrib = {}
  for word in top_words:
    atrib[f'contains({word})'] = (word in document_words)
  return atrib

In [ ]:
#probamos con uno
documento_atributos(df['tokens'].values[0])

In [ ]:
fset = [(documento_atributos(texto),clase) for texto, clase in zip(df['tokens'].values, df['clase'].values)]

In [ ]:
fset[1]

In [ ]:
random.shuffle(fset)

In [ ]:
train , test = train_test_split(fset, test_size=0.3)

In [ ]:
classifier = nltk.NaiveBayesClassifier.train(train)

In [ ]:
nltk.classify.accuracy(classifier, test)

0.49333333333333335

In [ ]:
#viendo atributos mas importantes
classifier.show_most_informative_features(10)

Most Informative Features
  contains(('has', 195)) = False              -1 : 1      =      1.0 : 1.0
  contains(('name', 67)) = False              -1 : 1      =      1.0 : 1.0
  contains(('days', 40)) = False              -1 : 1      =      1.0 : 1.0
    contains(('?', 360)) = False              -1 : 1      =      1.0 : 1.0
 contains(('there', 86)) = False              -1 : 1      =      1.0 : 1.0
    contains(('-', 283)) = False              -1 : 1      =      1.0 : 1.0
    contains(('@', 128)) = False              -1 : 1      =      1.0 : 1.0
    contains(('a', 879)) = False              -1 : 1      =      1.0 : 1.0
  contains(('made', 38)) = False              -1 : 1      =      1.0 : 1.0
contains(('because', 53)) = False              -1 : 1      =      1.0 : 1.0


In [ ]:
#ejercicio para filtrar documentos
from os import listdir
from zipfile import ZipFile

In [ ]:
def ls(ruta, type_data = ''):
    directory = [ruta+'//'+arch for arch in listdir(ruta) if  arch.endswith(type_data)]
    return directory

def unzip(file,path):
  ZipFile(file, 'r').extractall(path)

In [ ]:
path = 'datasets/email/plaintext/'
path_f='dataset_extraido/'

In [ ]:
for arch in ls(path, '.zip'): unzip(arch,path+path_f)

In [ ]:
data = [d for dir in ls(path+path_f) for d in ls(dir) if '.txt' not in d]

In [ ]:
spam,ham = 'spam','ham'

In [ ]:
def read_data(dir):
  d = open(dir, 'r', errors='ignore')
  return d.read()

In [ ]:
stop_words_en = set(stopwords.words('english'))

In [ ]:
stop_words_en

In [ ]:
def remove_stopwords(text):
  no_garbage = [w for w in text if not w in stop_words_en and not w in string.punctuation and len(w)>2]
  return no_garbage


In [ ]:
spam_data = nltk.FreqDist([t for dir in data for m in ls(dir) 
                           if spam in m for t in remove_stopwords(word_tokenize(read_data(m)))])

In [ ]:
top_words_spam = spam_data.most_common(5000)
top_words_spam[:200]

[('Subject', 26846),
 ('com', 18250),
 ('company', 18068),
 ('http', 13511),
 ('email', 12845),
 ('information', 12473),
 ('please', 11640),
 ('statements', 10599),
 ('may', 9961),
 ('one', 9567),
 ('get', 9437),
 ('business', 9148),
 ('money', 9078),
 ('000', 9008),
 ('time', 8902),
 ('new', 8561),
 ('www', 8324),
 ('mail', 7792),
 ('within', 7582),
 ('free', 7506),
 ('price', 7465),
 ('report', 7173),
 ('account', 6219),
 ('investment', 6169),
 ('click', 6124),
 ('make', 5982),
 ('future', 5718),
 ('message', 5705),
 ('inc', 5657),
 ('online', 5652),
 ('software', 5644),
 ('best', 5524),
 ('securities', 5516),
 ('like', 5492),
 ('stock', 5452),
 ('forward', 5389),
 ('need', 5254),
 ('adobe', 5238),
 ('looking', 5138),
 ('want', 5085),
 ('address', 5068),
 ('order', 5056),
 ('use', 4972),
 ('security', 4878),
 ('number', 4870),
 ('news', 4864),
 ('net', 4855),
 ('million', 4795),
 ('2005', 4777),
 ('site', 4754),
 ('would', 4731),
 ('also', 4680),
 ('2004', 4611),
 ('contact', 4494),


In [ ]:
ham_data = [t for dir in data for m in ls(dir) if spam in m for t in remove_stopwords(word_tokenize(read_data(m)))]

In [ ]:
def document_atrib(document):
  document_words = set(document)
  atrib = {}
  for word in top_words_spam:
    atrib[f'contains({word})'] = (word in document_words)
  return atrib